In [1]:
%run /eos/project/l/liu/Toolbox/myToolbox.py

%config InlineBackend.figure_format = 'retina'

Version 0.03. This is the latest version.
Please help me to improve it reporting bugs to guido.sterbini@cern.ch.


/cvmfs/sft.cern.ch/lcg/views/LCG_91/x86_64-slc6-gcc62-opt/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


Your platform is Linux-3.10.0-693.11.6.el7.x86_64-x86_64-with-redhat-6.9-Carbon
Your folder is /eos/user/a/ahuschau/Simulations/PS/optics_repository/iOMC/psmod/2018/output/knobs/LEQ
Your IP is 172.17.0.24
2018-06-04 14:04:34


In [2]:
folder = '/eos/user/a/ahuschau/Simulations/PS/optics_repository/iOMC/psmod/2018/output/knobs/LEQ/'
files = sorted(glob.glob(folder + '*.ks'))
files

['/eos/user/a/ahuschau/Simulations/PS/optics_repository/iOMC/psmod/2018/output/knobs/LEQ/Q_nominal.ks',
 '/eos/user/a/ahuschau/Simulations/PS/optics_repository/iOMC/psmod/2018/output/knobs/LEQ/dQx_negative.ks',
 '/eos/user/a/ahuschau/Simulations/PS/optics_repository/iOMC/psmod/2018/output/knobs/LEQ/dQx_positive.ks',
 '/eos/user/a/ahuschau/Simulations/PS/optics_repository/iOMC/psmod/2018/output/knobs/LEQ/dQy_negative.ks',
 '/eos/user/a/ahuschau/Simulations/PS/optics_repository/iOMC/psmod/2018/output/knobs/LEQ/dQy_positive.ks']

In [3]:
myDataFrame = pnd.DataFrame(columns = ['Qx', 'Qy', 'kF', 'kD'])

for i,f in enumerate(files):
    aux = pnd.read_csv(f, header=None)
    aux2 = aux[0].str.split().apply(lambda x: np.float(x[2])).to_frame().T
    aux2.columns = ['Qx', 'Qy', 'kF', 'kD']
    aux2 = aux2.rename(index={0: i})
    myDataFrame = myDataFrame.append(aux2)
    
myDataFrame

,Qx,Qy,kF,kD
0,0.2454,0.2837,0.000000,0.000000
1,0.1954,0.2837,-0.002097,0.001180
2,0.2954,0.2837,0.002049,-0.001134
3,0.2454,0.2337,-0.001182,0.002121
4,0.2454,0.3337,0.001137,-0.002080


In [4]:
myDataFrame['dQx'] = myDataFrame['Qx']-myDataFrame['Qx'].iloc[0]
myDataFrame['dQy'] = myDataFrame['Qy']-myDataFrame['Qy'].iloc[0]
myDataFrame

,Qx,Qy,kF,kD,dQx,dQy
0,0.2454,0.2837,0.000000,0.000000,0.00,0.00
1,0.1954,0.2837,-0.002097,0.001180,-0.05,0.00
2,0.2954,0.2837,0.002049,-0.001134,0.05,0.00
3,0.2454,0.2337,-0.001182,0.002121,0.00,-0.05
4,0.2454,0.3337,0.001137,-0.002080,0.00,0.05


# Knob computation

In [32]:
knobLEQ = pnd.DataFrame(index = [0], columns = ['dkF_H', 'dkD_H', 'dkF_V', 'dkD_V'])

In [33]:
#horizontal tune knob
knobLEQ['dkF_H'] = (myDataFrame['kF'][2]-myDataFrame['kF'][1])/(myDataFrame['dQx'][2]-myDataFrame['dQx'][1])
knobLEQ['dkD_H'] = (myDataFrame['kD'][2]-myDataFrame['kD'][1])/(myDataFrame['dQx'][2]-myDataFrame['dQx'][1])

print('dkF_H = ' + str(dkF_H))
print('dkD_H = ' + str(dkD_H))

dkF_H = 0.04146850823
dkD_H = -0.02313765143


In [34]:
#vertical tune knob
knobLEQ['dkF_V'] = (myDataFrame['kF'][4]-myDataFrame['kF'][3])/(myDataFrame['dQy'][4]-myDataFrame['dQy'][3])
knobLEQ['dkD_V'] = (myDataFrame['kD'][4]-myDataFrame['kD'][3])/(myDataFrame['dQy'][4]-myDataFrame['dQy'][3])

print('dkF_V = ' + str(dkF_V))
print('dkD_V = ' + str(dkD_V))

dkF_V = 0.02318755982
dkD_V = -0.0420109205


In [35]:
knobLEQ.to_pickle('knob_LEQ.pkl')

In [48]:
def knob_LEQ(dQ_H, dQ_V):
    knobLEQ = pnd.read_pickle('knob_LEQ.pkl')
    kd = dQ_H * knobLEQ['dkD_H'] + dQ_V * knobLEQ['dkD_V']
    kf = dQ_H * knobLEQ['dkF_H'] + dQ_V * knobLEQ['dkF_V']
    return kf[0], kd[0]
    

In [61]:
dQ_H = -0.05

knob_LEQ(0, dQ_H)

(-0.0011593779910000002, 0.0021005460250000003)

# Tune reconstruction from knob

In [62]:
def knob_Q(kf, kd):
    knobLEQ = pnd.read_pickle('knob_LEQ.pkl')
    dQ_H = (kf - kd * knobLEQ['dkF_V']/knobLEQ['dkD_V']) / (knobLEQ['dkF_H'] - knobLEQ['dkD_H']/knobLEQ['dkD_V']*knobLEQ['dkF_V'])
    dQ_V = (kf - kd * knobLEQ['dkF_H']/knobLEQ['dkD_H']) / (knobLEQ['dkF_V'] - knobLEQ['dkD_V']/knobLEQ['dkD_H']*knobLEQ['dkF_H'])
    return dQ_H[0], dQ_V[0]

In [64]:
knob_Q(-0.0011593779910000002, 0.0021005460250000003)

(0.0, -0.05000000000000001)